In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [3]:
import os
from google.colab import userdata
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')


In [4]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
data_analyst_agent = Agent(
    role = "Data Analyst",
    goal = "Monitor and analyze market data in real-time to identify trends and predict market movements.",
    backstory = "Specializing in financial markets, this agent employs statistical modeling and machine learning techniques to provide critical critical insights. Renowned for it's proficiency in data analysis, the data analyst agent serves as a pivotal resource for informing trading decision ",
    tools = [search_tool, scrape_tool],
    verbose = True,
    allow_delegation  = True
)

In [6]:
trading_strategy_agent = Agent(
    role = "Trading Strategy Developer",
    goal = "Develop and Test various trading strategies leveraging insights from Data Analyst agent.",
    backstory = "Possessing a deep understanding of financial markets and quantitative analysis, this agent formulates and optimizes trading strategies. It assesses the performance of diverse approaches to identify the most profitable and risk-averse options.",
    tools = [search_tool, scrape_tool],
    verbose = True,
    allow_delegation  = True
)

In [7]:
execution_agent = Agent(
    role = "Trade Advisor",
    goal = "Recommend optimal trade execution strategies based on approved trading plans.",
    backstory = "Specializing in the analysis of timing, price, and logistical details of potential trades, this agent evaluates these factors to provide well-founded recommendations. Its expertise ensures that trades are executed efficiently and in alignment with the overall strategy.",
    tools = [search_tool, scrape_tool],
    verbose = True,
    allow_delegation  = True
)

In [8]:
risk_management_agent = Agent(
    role = "Risk Advisor",
    goal = "Evaluate and provide insights on the risks associated with potential trading activities.",
    backstory = "With extensive expertise in risk assessment models and market dynamics, this agent thoroughly examines the potential risks of proposed trades. It delivers comprehensive analyses of risk exposure and recommends safeguards to ensure that trading activities align with the firm's risk tolerance.",
    tools = [search_tool, scrape_tool],
    verbose = True,
    allow_delegation  = True
)

#### Create Tasks

In [9]:
data_analysis_task = Task(
    description = (
        "Continuously monitor and analyze market data for the selected stock ({stock_selection}). "
        "Employ statistical modeling and machine learning techniques to identify trends and predict market movements.")
    ,
    expected_output = (
        "Generate insights and alerts regarding significant market opportunities or threats for {stock_selection}."),
    agent = data_analyst_agent
)

In [10]:
#Task for Trading Strategy Agent:Develop Trading Strategies
strategy_development_task  = Task(
    description = (
    "Develop and refine trading strategies based on insights from the Data Analyst and user-defined risk tolerance ({risk_tolerance}). "
        "Incorporate trading preferences ({trading_strategy_preference}) in the strategy development process.")
    ,
    expected_output = (
        "A set of potential trading strategies for {stock_selection} that align with the user's risk tolerance."),
    agent = trading_strategy_agent
)

In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the optimal execution methods for {stock_selection}, "
        "considering current market conditions and pricing strategies."
    ),
    expected_output=(
        "Comprehensive execution plans detailing how and when to execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks and recommend mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)


In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)


In [16]:
#Example data for kicking off the process
financial_trading_inputs = {
    "stock_selection": "NVDA",
    "initial_capital": '100000',
    "risk_tolerance": "Medium",
    "trading_strategy_preference": "Day Trading",
    "news_inpact_consideration": True
}

In [ ]:
result = financial_trading_crew.kickoff(inputs = financial_trading_inputs)

Streaming output truncated to the last 5000 lines.


> Entering new CrewAgentExecutor chain...
I need to find recent articles and market trends related to NVDA (NVIDIA Corporation). I can do this by using the search the internet tool with the appropriate query.

Action: 
Search the internet

Action Input: 
{"search_query": "recent market trends and articles on NVDA"}
 


Search results: Title: NVIDIA Corporation (NVDA) Latest Stock News & Headlines
Link: https://finance.yahoo.com/quote/NVDA/news/
Snippet: Get the latest NVIDIA Corporation (NVDA) stock news and headlines to help you in your trading and investing decisions.
---
Title: Nvidia (NVDA) Stock Quotes, Company News And Chart Analysis
Link: https://www.investors.com/news/technology/nvidia-stock-nvda-stock-quote-analysis/
Snippet: Nvidia stock page get's you the latest news from the artificial intelligence and AI chip gaint, plus quotes and analysis.
---
Title: Nvidia Stock Drops. Why It Could Still Reclaim Top Spot Over ...
Link

In [ ]:
from Ipython.display import display, Markdown
Markdown(result)

In [ ]:
#Example data for kicking off the process
financial_trading_inputs = {
    "stock_selection": "AAPL",
    "initial_capital": '100000',
    "risk_tolerance": "Medium",
    "trading_strategy_preference": "Day Trading",
    "news_inpact_consideration": True
}

In [ ]:
result = financial_trading_crew.kickoff(inputs = financial_trading_inputs)